# WE'LL START WITH WEATHER DATA

In [16]:
import requests
import pandas as pd
from dateutil import parser, rrule
from datetime import datetime, time, date
import time



In [2]:
station = 'IEDINBUR6' # Edinburgh
data_raw = pd.read_csv('~/Documents/datasets/weather/{}_weather.csv'.format(station),index_col=0)
print(f'Dataset contains {data_raw.shape[0]} rows and {data_raw.shape[1]} columns')
print(f'Data types of the raw (uncleaned) data:\n{data_raw.dtypes}')

Dataset contains 101443 rows and 17 columns
Data types of the raw (uncleaned) data:
Time                        object
TemperatureC               float64
DewpointC                  float64
PressurehPa                float64
WindDirection               object
WindDirectionDegrees         int64
WindSpeedKMH               float64
WindSpeedGustKMH           float64
Humidity                     int64
HourlyPrecipMM             float64
Conditions                 float64
Clouds                     float64
dailyrainMM                float64
SolarRadiationWatts/m^2    float64
SoftwareType                object
DateUTC                     object
station                     object
dtype: object


In [3]:
# Sample look at the raw data
data_raw.head()


,Time,TemperatureC,DewpointC,PressurehPa,WindDirection,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,Conditions,Clouds,dailyrainMM,SolarRadiationWatts/m^2,SoftwareType,DateUTC,station
0,2015-01-01 00:00:00,10.3,4.5,1001.9,SW,236,9.0,11.9,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:00:00,IEDINBUR6
1,2015-01-01 00:06:00,10.2,4.4,1001.9,SW,226,6.9,7.9,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:06:00,IEDINBUR6
2,2015-01-01 00:11:00,10.2,4.4,1001.9,SSW,202,7.2,15.8,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:11:00,IEDINBUR6
3,2015-01-01 00:16:00,10.2,4.4,1001.9,SSW,202,7.9,11.9,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:16:00,IEDINBUR6
4,2015-01-01 00:21:00,10.2,4.6,1002.3,South,174,4.3,7.9,68,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:21:00,IEDINBUR6


In [4]:
# Give the variables some friendlier names and convert types as necessary.
data_raw['temp'] = data_raw['TemperatureC'].astype(float)
data_raw['rain'] = data_raw['HourlyPrecipMM'].astype(float)
data_raw['total_rain'] = data_raw['dailyrainMM'].astype(float)
data_raw['date'] = data_raw['DateUTC'].apply(parser.parse)
data_raw['humidity'] = data_raw['Humidity'].astype(float)
data_raw['wind_direction'] = data_raw['WindDirectionDegrees']
data_raw['wind'] = data_raw['WindSpeedKMH']


In [5]:
data_raw.head()

,Time,TemperatureC,DewpointC,PressurehPa,WindDirection,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,...,SoftwareType,DateUTC,station,temp,rain,total_rain,date,humidity,wind_direction,wind
0,2015-01-01 00:00:00,10.3,4.5,1001.9,SW,236,9.0,11.9,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:00:00,IEDINBUR6,10.3,0.0,0.0,2015-01-01 00:00:00,67.0,236,9.0
1,2015-01-01 00:06:00,10.2,4.4,1001.9,SW,226,6.9,7.9,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:06:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:06:00,67.0,226,6.9
2,2015-01-01 00:11:00,10.2,4.4,1001.9,SSW,202,7.2,15.8,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:11:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:11:00,67.0,202,7.2
3,2015-01-01 00:16:00,10.2,4.4,1001.9,SSW,202,7.9,11.9,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:16:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:16:00,67.0,202,7.9
4,2015-01-01 00:21:00,10.2,4.6,1002.3,South,174,4.3,7.9,68,0.0,...,HP1001 V1.1.3,2015-01-01 00:21:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:21:00,68.0,174,4.3


In [6]:
# Extract out only the data we need.
data = data_raw.loc[:, ['date', 'station', 'temp', 'rain', 'total_rain', 'humidity', 'wind']]
data = data[(data['date'] >= datetime(2015,1,1)) & (data['date'] <= datetime(2015,12,31))]
data.head()

,date,station,temp,rain,total_rain,humidity,wind
0,2015-01-01 00:00:00,IEDINBUR6,10.3,0.0,0.0,67.0,9.0
1,2015-01-01 00:06:00,IEDINBUR6,10.2,0.0,0.0,67.0,6.9
2,2015-01-01 00:11:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.2
3,2015-01-01 00:16:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.9
4,2015-01-01 00:21:00,IEDINBUR6,10.2,0.0,0.0,68.0,4.3


In [7]:
# There's an issue with some stations that record rainfall ~-2500 where data is missing.
if (data['rain'] < -500).sum() > 10:
    print("There's more than 10 messed up days for {}".format(station))
    
# remove the bad samples
data = data[data['rain'] > -500]

In [8]:
# Assign the "day" to every date entry
data['day'] = data['date'].apply(lambda x: x.date())

# Get the time, day, and hour of each timestamp in the dataset
data['time_of_day'] = data['date'].apply(lambda x: x.time())
data['day_of_week'] = data['date'].apply(lambda x: x.weekday())    
data['hour_of_day'] = data['time_of_day'].apply(lambda x: x.hour)
# Mark the month for each entry so we can look at monthly patterns
data['month'] = data['date'].apply(lambda x: x.month)


,date,station,temp,rain,total_rain,humidity,wind,day,time_of_day,day_of_week,hour_of_day,month
0,2015-01-01 00:00:00,IEDINBUR6,10.3,0.0,0.0,67.0,9.0,2015-01-01,00:00:00,3,0,1
1,2015-01-01 00:06:00,IEDINBUR6,10.2,0.0,0.0,67.0,6.9,2015-01-01,00:06:00,3,0,1
2,2015-01-01 00:11:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.2,2015-01-01,00:11:00,3,0,1
3,2015-01-01 00:16:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.9,2015-01-01,00:16:00,3,0,1
4,2015-01-01 00:21:00,IEDINBUR6,10.2,0.0,0.0,68.0,4.3,2015-01-01,00:21:00,3,0,1


In [33]:
# Is each time stamp on a working day (Mon-Fri)
data['working_day'] = (data['day_of_week'] >= 0) & (data['day_of_week'] <= 4)

# Classify into morning or evening times (assuming travel between 8.15-9am and 5.15-6pm)
data['morning'] = (data['hour_of_day'] >= 7) & (data['hour_of_day'] <= 9)
data['evening'] = (data['hour_of_day'] >= 17) & (data['hour_of_day'] <= 18)

# If there's any rain at all, mark that!
data['raining'] = data['rain'] > 0.0

# You get wet cycling if its a working day, and its raining at the travel times!
data['get_wet_cycling'] = (data['working_day']) & ((data['morning'] & data['rain']) |
                                                   (data['evening'] & data['rain']))


In [37]:
data.loc[(data['get_wet_cycling'] == True) ].head()

,date,station,temp,rain,total_rain,humidity,wind,day,time_of_day,day_of_week,hour_of_day,month,working_day,raining,morning,evening,get_wet_cycling
91,2015-01-01 08:00:00,IEDINBUR6,10.4,1.8,0.3,80.0,6.4,2015-01-01,08:00:00,3,8,1,True,True,True,False,True
92,2015-01-01 08:05:00,IEDINBUR6,10.3,1.8,0.3,82.0,6.9,2015-01-01,08:05:00,3,8,1,True,True,True,False,True
98,2015-01-01 08:36:00,IEDINBUR6,9.9,1.8,0.5,82.0,15.8,2015-01-01,08:36:00,3,8,1,True,True,True,False,True
99,2015-01-01 08:41:00,IEDINBUR6,9.9,1.8,0.5,83.0,6.4,2015-01-01,08:41:00,3,8,1,True,True,True,False,True
113,2015-01-01 09:53:00,IEDINBUR6,10.3,1.8,1.0,79.0,11.6,2015-01-01,09:53:00,3,9,1,True,True,True,False,True


# Data summarisation and aggregation

With the data cleansed, we now have non-uniform samples of the weather at a given station throughout the year, at a sub-hour level. To make meaningful plots on this data, we can aggregate over the days and months to gain an overall view and to compare across stations.

In [40]:
# Looking at the working days only and create a daily data set of working days:
wet_cycling = data[data['working_day'] == True].groupby('day')['get_wet_cycling'].any()
wet_cycling = pd.DataFrame(wet_cycling).reset_index()

# Group by month for display - monthly data set for plots.
wet_cycling['month'] = wet_cycling['day'].apply(lambda x: x.month)
monthly = wet_cycling.groupby('month')['get_wet_cycling'].value_counts().reset_index(drop=True)
monthly.rename(columns={"get_wet_cycling":"Rainy", 0:"Days"}, inplace=True)
monthly.replace({"Rainy": {True: "Wet", False:"Dry"}}, inplace=True)    
#monthly['month_name'] = monthly['month'].apply(lambda x: calendar.month_abbr[x])
monthly.head()

0    14
1     8
2    19
3     1
4    17
dtype: int64

In [42]:

# Get aggregate stats for each day in the dataset on rain in general - for heatmaps.
rainy_days = data.groupby(['day']).agg({
        "rain": {"rain": lambda x: (x > 0.0).any(),
                 "rain_amount": "sum"},
        "total_rain": {"total_rain": "max"},
        "get_wet_cycling": {"get_wet_cycling": "any"}
        })   

In [43]:
# clean up the aggregated data to a more easily analysed set:
rainy_days.reset_index(drop=False, inplace=True) # remove the 'day' as the index
rainy_days.rename(columns={"":"date"}, inplace=True) # The old index column didn't have a name - add "date" as name
rainy_days.columns = rainy_days.columns.droplevel(level=0) # The aggregation left us with a multi-index
                                                           # Remove the top level of this index.
rainy_days['rain'] = rainy_days['rain'].astype(bool)   